# N2O actinometry experiment | It-factor computation

In [10]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from scipy import integrate

##
# --N2O_flow: set N2O flow rate (sccm)
# --N2_flow: set N2 flow rate (sccm)
# --NOx_ppb: NOₓ concentration detected by the NOₓ monitor (ppb)
# --T: average of N2O flow and N2 flow temperatures (°C)
# --QtotNOx: inlet flow rate used in the actinometry experiment (lpm)
# --Qinstrument: inlet flow rate used in the sulfuric acid calibration experiment (lpm)
# --R: inner radius of quartz tube (cm)
# --monitor_bg: background NOₓ concentration (ppb) detected by the NOₓ monitor when N2O flow = 0 and Hg-lamp is on
# --bottle_bg: NOₓ concentration (ppb) detected by the NOₓ monitor repeating the experiment with Hg-lamp off


## user interface
N2O_flow = np.array([804,1034,1250,1452,1641,1818,1985,2143,2292,2432]) # sccm, ADJUST
N2_flow = np.array([6696,6466,6250,6048,5859,5682,5515,5357,5208,5068]) # sccm, ADJUST
NOx_ppb = np.array([4.325,5.650,6.749,7.642,8.609,9.410,10.060,10.657,11.241,11.581]) # ppb, ADJUST

T = 25 # °C, ADJUST

QtotNOx=7.5 # lpm, ADJUST
Qinstrument = 7.5  # lpm, ADJUST

R = 0.78  # 0.78 cm, ADJUST

monitor_bg=0.8; # ppb, ADJUST
bottle_bg=[0.4,0.2,0.2,0.1,0.08,0.06,0.06,0.04,0.04,0.02];  # ppb, ADJUST


## It-product non-corrected
T = T + 273.15  # K
p=96000 # Pa
kB=1.3806488e-23 # J K-1, Boltzmann constant

K6 = 2e-11 * np.exp(130 / T)  # cm3 mol-1 s-1
K7 = 7.6e-11  # cm3 mol-1 s-1
K8 = 4.3e-11  # cm3 mol-1 s-1
K9 = 6.0e-12  # cm3 mol-1 s-1

qyN2O = 1  # quantum yield
csN2O = 1.43e-19  # absorption cross section, cm2

M=p/(kB*T)/1e6
N2O_ratio = N2O_flow / (N2O_flow + N2_flow)
N2_ratio = N2_flow / (N2O_flow + N2_flow)
N2O=N2O_ratio * M; # cm-3
N2=N2_ratio * M; # cm-3
NOx_corrected_ppb = NOx_ppb - monitor_bg - bottle_bg # ppb
NOx = NOx_corrected_ppb*1e-9*M; # cm-3

ItNOx = (K6 * N2 + (K7 + K8 + K9) * N2O) * NOx / (2 * K7 * csN2O * qyN2O * (N2O ** 2)); # photons cm-2
It = ItNOx * (QtotNOx / Qinstrument) # photons cm-2

print('It', [f"{val:.2E}" for val in It])
print('\nIt_mean =', "{:.4E}".format(np.mean(It)))
print('It_median =', "{:.4E}".format(np.median(It)))

It ['5.13E+11', '4.94E+11', '4.44E+11', '4.07E+11', '3.83E+11', '3.60E+11', '3.37E+11', '3.20E+11', '3.06E+11', '2.89E+11']

It_mean = 3.8522E+11
It_median = 3.7117E+11


In [11]:
## It-product corrected

m = len(N2O_ratio)
fx = []
for i in range(m):
    f = lambda y, x: np.exp(-csN2O * N2O[i] * (np.sqrt(R ** 2 - y ** 2) + x))
    fx1 = integrate.dblquad(f, -R, R, lambda x: -np.sqrt(R ** 2 - x ** 2), lambda x: np.sqrt(R ** 2 - x ** 2))
    fx.append(fx1[0])
q = np.transpose(fx)
K = q / R ** 2 / np.pi

ItCorrected = It / K

It_corrected_mean = np.mean(ItCorrected)
It_corrected_median = np.median(ItCorrected)

print("K =", [f"{val:.4f}" for val in K])
print('\nIt_corrected', [f"{val:.4E}" for val in ItCorrected])
print('\nIt_corrected_mean =', "{:.4E}".format(It_corrected_mean))
print('It_corrected_median =', "{:.4E}".format(It_corrected_median))

K = ['0.7977', '0.7507', '0.7101', '0.6750', '0.6445', '0.6178', '0.5941', '0.5729', '0.5540', '0.5371']

It_corrected ['6.4323E+11', '6.5820E+11', '6.2496E+11', '6.0245E+11', '5.9364E+11', '5.8226E+11', '5.6781E+11', '5.5818E+11', '5.5192E+11', '5.3851E+11']

It_corrected_mean = 5.9212E+11
It_corrected_median = 5.8795E+11
